# IMPORTS

In [51]:
import pandas as pd
import numpy as np
import random
from  math import log2
from collections import Counter
from cadsgen import generate_payload_agri_parcel_record
from entropies import shannon_entropy_kde
from shapley import sampling_kg
from scipy.stats import gaussian_kde
from scipy.stats import entropy
from sklearn.preprocessing import minmax_scale as scale
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tqdm

np.random.seed(7)

# DECLARATION DES FONCTIONS

In [9]:
def kseuil(m:int = 100, epsilon:float = 0.1, sigma:float = 0.1) :
    """_summary_
    Args:
        m (nombre de joueurs, optional): _description_. Defaults to 100.
        epsilon (précision, optional): _description_. Defaults to 0.1.
        sigma (erreur de type I, optional): _description_. Defaults to 0.1.

    Returns:
        _int_: _nombre minimale d'échantillonnage pour avoir |shap_approx - shap| < epsilon avec la certitude 1 - sigma_
    """
    return int((m*np.log(2/sigma))/(2*(epsilon**2)))

def genexpo_distxp(params:dict) :
    np.random.seed(7)
    data = {'id':[],
            'dist':[],
            'e_sqrt':[],
            'e_doane':[],
            'e_auto':[],
            'mu':[],
            'med':[],
            'std':[],
            'taille':[],
            'bins_sqrt':[],
            'bins_doane':[],
            'bins_auto':[]}
    id = 0
    
    for t in tqdm.tqdm(params['taille']) :
        for l in params['lambda'] :
            id = id + 1
            data['id'].append(f"exp_{id}")
            data['taille'].append(t)

            sample = np.random.exponential(scale=1/l, size=t)
            data['mu'].append(np.mean(sample))
            data['med'].append(np.median(sample))
            data['std'].append(np.std(sample))

            freq = np.histogram(sample, bins='auto', density=False)[0]/t
            data['e_auto'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
            data['bins_auto'].append(freq.shape[0])

            freq = np.histogram(sample, bins='sqrt', density=False)[0]/t
            data['e_sqrt'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
            data['bins_sqrt'].append(freq.shape[0])

            freq = np.histogram(sample, bins='doane', density=False)[0]/t
            data['e_doane'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
            data['bins_doane'].append(freq.shape[0])

    data['dist'] = ['exp']*len(data['id'])
    return data

def gennorm_distxp(params:dict) :
    np.random.seed(7)
    data = {'id':[],
            'dist':[],
            'e_sqrt':[],
            'e_doane':[],
            'e_auto':[],
            'mu':[],
            'med':[],
            'std':[],
            'taille':[],
            'bins_sqrt':[],
            'bins_doane':[],
            'bins_auto':[]}
    id = 0
    
    for t in tqdm.tqdm(params['taille']) :
        for m in range(len(params['mu'])):
            for s in params['sigma'][m] :
                id = id + 1
                data['id'].append(f"norm_{id}")
                data['taille'].append(t)

                sample = np.random.normal(loc=m, scale=s, size=t)
                data['mu'].append(np.mean(sample))
                data['med'].append(np.median(sample))
                data['std'].append(np.std(sample))
            

                freq = np.histogram(sample, bins='auto', density=False)[0]/t
                data['e_auto'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_auto'].append(freq.shape[0])

                freq = np.histogram(sample, bins='sqrt', density=False)[0]/t
                data['e_sqrt'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_sqrt'].append(freq.shape[0])

                freq = np.histogram(sample, bins='doane', density=False)[0]/t
                data['e_doane'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_doane'].append(freq.shape[0])

    data['dist'] = ['norm']*len(data['id'])
    return data

def genuni_distxp(params:dict) :
    np.random.seed(7)
    data = {'id':[],
            'dist':[],
            'e_sqrt':[],
            'e_doane':[],
            'e_auto':[],
            'mu':[],
            'med':[],
            'std':[],
            'taille':[],
            'bins_sqrt':[],
            'bins_doane':[],
            'bins_auto':[]}
    id = 0
    
    for t in tqdm.tqdm(params['taille']):
        for c in range(len(params['center'])):
            for r in params['range'][c] :
                id = id + 1
                data['id'].append(f"uni_{id}")
                data['taille'].append(t)

                sample = np.random.uniform(low=(c-r), high=(c+r), size=t)
                data['mu'].append(np.mean(sample))
                data['med'].append(np.median(sample))
                data['std'].append(np.std(sample))
            

                freq = np.histogram(sample, bins='auto', density=False)[0]/t
                data['e_auto'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_auto'].append(freq.shape[0])

                freq = np.histogram(sample, bins='sqrt', density=False)[0]/t
                data['e_sqrt'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_sqrt'].append(freq.shape[0])

                freq = np.histogram(sample, bins='doane', density=False)[0]/t
                data['e_doane'].append(entropy(freq, base = 2)/np.log2(freq.shape[0]))
                data['bins_doane'].append(freq.shape[0])

    data['dist'] = ['uni']*len(data['id'])
    return data

def scaler(contributions: dict) :
    """_summary_

    Args:
        contributions (dict): _description_
    """
    binf = min(contributions.values())
    bsup = max(contributions.values())
    cont_scaled = {item: (value - binf) / (bsup - binf) for item, value in contributions.items()}
    return cont_scaled

def sigmoid(x : float, m:float = 1):
    """_summary_

    Args:
        x (float): _description_
        m (float, optional): _description_. Defaults to 1.

    Returns:
        _type_: _description_
    """
    return 1 / (1 + np.exp(-m*x))

def get_scores(cont_scaled:dict, spread:float =1) -> dict :
    """_summary_

    Args:
        cont_scaled (dict): _description_

    Returns:
        dict: _description_
    """
    scores = {item: sigmoid(value,m=spread) for item, value in cont_scaled.items()}
    
    return scores

def get_weights(scores:dict) -> dict:
    """_summary_

    Args:
        scores (dict): _description_

    Returns:
        dict: _description_
    """
    weights = {item:round(100*value/sum(scores.values()),3) for item, value in scores.items()}
    
    return weights

def gennorm_wmissing(samples:int = 30, mu:float = 0.0, std:float = 1.0, nanratio:float = 0.2):
    """_summary_

    Args:
        size (int, optional): _Nombre total d'échantillons_. Defaults to 30.
        mu (float, optional): _Moyenne de la distribution normale_. Defaults to 0.0.
        std (float, optional): _Écart-type de la distribution normale_. Defaults to 1.0.
        nanratio (float, optional): _Proportion de données manquantes_. Defaults to 0.2.
    Returns:
        _array_: _generated samples with specified missing values_
    """
    data = np.random.normal(loc=mu, scale=std, size=samples)
    missing_sample_size = int(samples*nanratio)
    missing_indices = np.random.choice(samples, missing_sample_size, replace=False)
    data[missing_indices] = np.nan
    return data

def delete_values(sample:np.array, hist:tuple, ratio:float, size:int):
    array = sample.copy()
    to_delete = int(ratio*size)
    nb_to_delete_by_range = [0]*hist[0].shape[0]
    rg_sorted = np.argsort(hist[0]).tolist()
    i = 0
    while to_delete > 0 and i < len(rg_sorted):
        if hist[0][i] > 1:
            nb_to_delete = min(hist[0][i] - 1, to_delete)
            nb_to_delete_by_range[i] = nb_to_delete
            to_delete -= nb_to_delete
        i += 1
    for rg in rg_sorted :
        value_to_delete = np.random.choice(array[(array >= hist[1][rg]) & (array <= hist[1][rg+1])], size=nb_to_delete_by_range[rg], replace=False)
        array = np.delete(array, np.where(np.isin(array, value_to_delete)))
    
    return array

def gennorm_dropxp(params:dict) :
    np.random.seed(7)
    data = {'id':[],
            'dist':[],
            'e':[],
            'alt_e':[],
            'taille':[],
            'droped_ratio':[],
            'bins':[]}
    
    for t in tqdm.tqdm(params['taille']):
        for rat in params['ratio'] :
            data['id'].append(f"norm_{t}_{0}_{round(1,2)*1000}")
            data['taille'].append(t)            
            sample = np.random.normal(loc=0, scale=1, size=t)
            hist = np.histogram(sample, bins='doane', density=False)
            freq = hist[0]/t
            bin = freq.shape[0]
            data['bins'].append(bin)
            data['baseh'].append(entropy(freq, base = 2))

            alter_sample = delete_values(sample, hist, rat, t)
            alter_freq = np.histogram(alter_sample, bins=hist[1], density=False)[0]/alter_sample.shape[0]
            data['alterh'] .append(entropy(alter_freq, base = 2))
            data['droped_ratio'] .append(rat)

    data['dist'] = ['norm']*len(data['id'])
    return data

# PARAMETRES

In [28]:
normparams = {'taille': [15, 30, 60, 120, 240, 480, 960, 1920, 4000, 6000, 8000,10000],
          'mu':[0, 5, 15, 35, 75, 155],
          'sigma':[np.linspace(0.1, 1, 10),
                   np.linspace(0.1, 1, 10),
                   np.linspace(0.1, 3, 10),
                   np.linspace(0.1, 7, 10),
                   np.linspace(0.1, 15, 10),
                   np.linspace(0.1, 31, 10)]}

uniparams = {'taille': [15, 30, 60, 120, 240, 480, 960, 1920, 4000, 6000, 8000,10000],
          'center':[0, 5, 15, 35, 75, 155],
          'range':[np.linspace(0.1, 1, 10),
                   np.linspace(0.1, 1, 10),
                   np.linspace(0.1, 3, 10),
                   np.linspace(0.1, 7, 10),
                   np.linspace(0.1, 15, 10),
                   np.linspace(0.1, 31, 10)]}

expoparams = {'taille': [15, 30, 60, 120, 240, 480, 960, 1920, 4000, 6000, 8000,10000],
          'lambda':np.linspace(0.0064, 1000, 6).tolist()}

# DATAFRAMES

In [29]:
distxp = pd.concat([pd.DataFrame(genexpo_distxp(expoparams)),
                    pd.DataFrame(gennorm_distxp(normparams)),
                    pd.DataFrame(genuni_distxp(uniparams))])

100%|██████████| 12/12 [00:00<00:00, 22.80it/s]


# HYPOTHESES

## CALCUL DES CONTRIBUTION

### **JEUX COLLABORATIF**
> Comme nous somme en présence d'une situation de mise en commun de plusieurs jeux de données pour la création d'un autre qui va générer de la valeur nous pouvons modéliser la situation par un jeu collaboatif et utiliser les outils de ce domaine de la théorie des jeux.

#### VALEUR DE SHAPLEY
> La valeur de shapley est très adopté pour le calcul des contributions dans le cadre de jeux collaboratifs. C'est le seul outil à notre connaissance qui permet équité dans le partage de revenus.
>- Il devrait être possible de calculer les contributions avec la valeur de Shapley.

## **VALEUR DE LA CONNAISSANCE**

#### L'entropie de shanon comme mesure de la qualité d'une donnée
> Etant données que des métadonnées existent pour les données pour permettre aux acheter d'évaluer leur pertinence (colonnes, taille, ancienneté, etc.), l'asymétrie d'information se situe sur la structure des données et les données manquantes.
> Nous pensons que l'entropie de shannon (indice de shannon en biodiversité) peut être un excellent indicateur de la distribution dans les données et servir de base pour leurs tarifications. Toutefois, il exite un certain nombre de choses à vérifier pour valider cette hypothèses.

##### Facteurs qui influencent l'entropie

>- dépend de la taille des jeux de données
>- Dépend de la distribution de probabilité
>- Dépend de la méthode de calcul des classes
>- Dépend de l'hétérogénéité des données 
>- de l'équilibre dans la distribtution des données (uniformité)

##### Comment affecter l'entropie
>La valeur de la connaissance devant être une fonction croissante de l'information apporté, Il s'agit ici de touver la stratégie qui permettent de réduire cette information apporté linéairement à l'écart entre le prix offre et le prix demande si cela est possible.
>- destruction d'information de sorte à accentuer le déséquilibre => supprimer des valeurs dans les classes de faibles fréquence

#### Comment les données manquantes peuvent affecter l'entropie
>les donnés manquantes techniquement ne peuvent affecter. On va parler de penaliser.
>l'entropie étant la mesure de la qualité, il est important que les données qui contiennent des données manquantes voient leurs entropies penalisés.
>- Une donnée manquante n'apporte pas d'information
>- On ne sait pas ce que aurait pu être cette donnée manquante 
>- imputer les données manquante de la moyenne c'est améliorer l'entropie ?
    - 20 % de données manquantes => la classe de la moyenne obtiens 20 % de données en plus.

# TEST D'HYPOTHESES

## VALEUR DE LA CONNAISSANCE

#### *Facteurs qui influencent l'entropie*

##### La taille

###### Normale x uniform x expo

In [ ]:
# expFORM - ENTROPIE X TAILLE- METHODE DOANE
x = distxp['taille']
y = distxp['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE DOANE
x = distxp['bins_doane']
y = distxp['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Nombre de classes - Méthode DOANE",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X TAILLE- METHODE AUTO
x = distxp['taille']
y = distxp['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE AUTO
x = distxp['bins_auto']
y = distxp['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Nombre de classes - Méthode AUTO",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X TAILLE- METHODE SQRT
x = distxp['taille']
y = distxp['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE SQRT
x = distxp['bins_sqrt']
y = distxp['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon",
                  xaxis_title= "Nombre de classes - Méthode SQRT",
                  yaxis_title= "Entropie")
fig.show()

###### Normale

In [47]:
# NORMALE - ENTROPIE X TAILLE- METHODE DOANE
x = distxp[distxp['dist']=='norm']['taille']
y = distxp[distxp['dist']=='norm']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()
# NORMALE - ENTROPIE X NOMBRE DE CLASSES - METHODE DOANE
x = distxp[distxp['dist']=='norm']['bins_doane']
y = distxp[distxp['dist']=='norm']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Nombre de classes - Méthode Doane",
                  yaxis_title= "Entropie")
fig.show()

# NORMALE - ENTROPIE X TAILLE- METHODE SQRT
x = distxp[distxp['dist']=='norm']['taille']
y = distxp[distxp['dist']=='norm']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()

# NORMALE - ENTROPIE X NOMBRE DE CLASSES - METHODE SQRT
x = distxp[distxp['dist']=='norm']['bins_sqrt']
y = distxp[distxp['dist']=='norm']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Nombre de classes - Méthode SQRT",
                  yaxis_title= "Entropie")
fig.show()

# NORMALE - ENTROPIE X TAILLE- METHODE AUTO
x = distxp[distxp['dist']=='norm']['taille']
y = distxp[distxp['dist']=='norm']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()

# NORMALE - ENTROPIE X NOMBRE DE CLASSES - METHODE AUTO
x = distxp[distxp['dist']=='norm']['bins_auto']
y = distxp[distxp['dist']=='norm']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Normale",
                  xaxis_title= "Nombre de classes - Méthode AUTO",
                  yaxis_title= "Entropie")
fig.show()

###### UNIFORM

In [48]:

# UNIFORM - ENTROPIE X TAILLE- METHODE DOANE
x = distxp[distxp['dist']=='uni']['taille']
y = distxp[distxp['dist']=='uni']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# UNIFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE DOANE
x = distxp[distxp['dist']=='uni']['bins_doane']
y = distxp[distxp['dist']=='uni']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Nombre de classes - Méthode DOANE",
                  yaxis_title= "Entropie")
fig.show()

# UNIFORM - ENTROPIE X TAILLE- METHODE AUTO
x = distxp[distxp['dist']=='uni']['taille']
y = distxp[distxp['dist']=='uni']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()

# UNIFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE AUTO
x = distxp[distxp['dist']=='uni']['bins_auto']
y = distxp[distxp['dist']=='uni']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Nombre de classes - Méthode AUTO",
                  yaxis_title= "Entropie")
fig.show()

# UNIFORM - ENTROPIE X TAILLE- METHODE SQRT
x = distxp[distxp['dist']=='uni']['taille']
y = distxp[distxp['dist']=='uni']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# UNIFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE SQRT
x = distxp[distxp['dist']=='uni']['bins_sqrt']
y = distxp[distxp['dist']=='uni']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Uniforme",
                  xaxis_title= "Nombre de classes - Méthode SQRT",
                  yaxis_title= "Entropie")
fig.show()

###### EXPONENTIEL

In [49]:
# expFORM - ENTROPIE X TAILLE- METHODE DOANE
x = distxp[distxp['dist']=='exp']['taille']
y = distxp[distxp['dist']=='exp']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE DOANE
x = distxp[distxp['dist']=='exp']['bins_doane']
y = distxp[distxp['dist']=='exp']['e_doane']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Nombre de classes - Méthode DOANE",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X TAILLE- METHODE AUTO
x = distxp[distxp['dist']=='exp']['taille']
y = distxp[distxp['dist']=='exp']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE AUTO
x = distxp[distxp['dist']=='exp']['bins_auto']
y = distxp[distxp['dist']=='exp']['e_auto']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Nombre de classes - Méthode AUTO",
                  yaxis_title= "Entropie")
fig.show()

# expFORM - ENTROPIE X TAILLE- METHODE SQRT
x = distxp[distxp['dist']=='exp']['taille']
y = distxp[distxp['dist']=='exp']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Taille de l'échantillon de base",
                  yaxis_title= "Entropie")
fig.show()


# expFORM - ENTROPIE X NOMBRE DE CLASSES- METHODE SQRT
x = distxp[distxp['dist']=='exp']['bins_sqrt']
y = distxp[distxp['dist']=='exp']['e_sqrt']
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers')])
fig.update_layout(title="Variation de l'entropie de Shannon - Distribtion Exponentielle",
                  xaxis_title= "Nombre de classes - Méthode SQRT",
                  yaxis_title= "Entropie")
fig.show()

##### Distribution de probabilité

##### Methode de calcul du nombres classes

#### *Comment affecter l'entropie*

##### Destruction de données

#### *Comment penaliser les données manquantes*

##### Imputer les nan de la valeur centrale de la classe qui à le plus effectif

# BROUILLON

In [ ]:
# Données
data = pd.read_csv("expdf_hist.csv")  # Remplacez "data.csv" par le nom de votre fichier de données

# Variables qualitatives
qualitatives = ['C(dist)']

# Variables quantitatives
quantitatives = ['basestd','basemu','basemed', 'bins'] #, 'basemu','basemed'

# Variable dépendante
dependante = 'e_base'

# Construction du modèle
formula = dependante + ' ~ ' + ' * '.join(qualitatives + quantitatives)
model = ols(formula, data=data).fit()

# Affichage des résultats
print(model.summary())